<a href="https://colab.research.google.com/github/ChloeZhou1997/BreastCancerCNN/blob/main/Breast_Cancer_Classfication_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pylibjpeg==1.1.1
!pip install pydicom==2.1.1
!pip install torchmetrics
!pip install pytorch_lightning
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

Mounted at /content/drive
     |████████████████████████████████| 1.3 MB 21.7 MB/s 
     |████████████████████████████████| 1.9 MB 14.0 MB/s 
     |████████████████████████████████| 408 kB 12.5 MB/s 
     |████████████████████████████████| 582 kB 13.7 MB/s 
     |████████████████████████████████| 136 kB 76.4 MB/s 
     |████████████████████████████████| 596 kB 60.5 MB/s 
     |████████████████████████████████| 1.1 MB 53.4 MB/s 
     |████████████████████████████████| 271 kB 77.4 MB/s 
     |████████████████████████████████| 144 kB 76.5 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import random

In [3]:
normalizer = np.load('/content/drive/MyDrive/Project/normalizer.npy')
mean = normalizer[0]
std = normalizer[1]

# Data Loading

In [4]:
import torch
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
# from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm.notebook import tqdm
import numpy as np

In [6]:
def load_file(path):
  return np.load(path).astype(np.float32)

In [7]:
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean,std),
                                       ]
)

val_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean,std),                                     
])

In [8]:
train_dataset = torchvision.datasets.DatasetFolder("/content/drive/MyDrive/Data/processed/training",
                                                   loader = load_file,
                                                   extensions = "npy",
                                                   transform = train_transforms)

val_dataset = torchvision.datasets.DatasetFolder("/content/drive/MyDrive/Data/processed/validation",
                                                   loader = load_file,
                                                   extensions = "npy",
                                                   transform = train_transforms)

In [9]:
batch_size = 8

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

#Model Creation

In [10]:
from torchvision.models.googlenet import BasicConv2d
model = torchvision.models.inception_v3()
model.AuxLogits.fc

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:48: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


Linear(in_features=768, out_features=1000, bias=True)

In [11]:
import torchmetrics

class CheastCancer_ConvNet(pl.LightningModule):

  def __init__(self,init_weights=True):
    super().__init__()

    self.model = torchvision.models.inception_v3()
    self.model.Conv2d_1a_3x3 = BasicConv2d(1, 32, kernel_size=3, stride=1)
    # self.model.Mixed_5b.branch5x5_2 = BasicConv2d(48, 64, kernel_size=3, padding=1)
    self.model.fc = torch.nn.Linear(in_features=2048, out_features=1, bias=True)
    # self.model.AuxLogits.fc = torch.nn.Linear(in_features=768, out_features=1, bias=True)
    
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr = 1e-3)

    #loss function
    self.criterion = torch.nn.BCEWithLogitsLoss()

    #eval matrix
    self.train_acc = torchmetrics.Accuracy()
    self.val_acc = torchmetrics.Accuracy()


  def forward(self, data):
    pred = self.model(data)
    return pred

  def training_step(self, batch, batch_idx):
    img, label = batch
    pred = self(img)
    # label = label.float()
    loss = self.criterion(pred.logits,label.float().unsqueeze(1))

    self.log("Train Loss", loss)
    self.log("Step Train ACC", self.train_acc(torch.sigmoid(pred.logits), label.int().unsqueeze(1)))

    return loss
  
  def training_epoch_end(self, outs):
    self.log("Train ACC", self.train_acc.compute())

  def validation_step(self, batch, batch_idx):
    img, label = batch
    # label = label.float()
    pred = self(img)
    # loss = self.loss_fn(pred,label)
    if isinstance(pred, tuple):
      loss = self.criterion(pred.logits,label.float().unsqueeze(1))
    else:
      loss = self.criterion(pred,label.float().unsqueeze(1))

    self.log("Train Loss", loss)
    self.log("Step Train ACC", self.val_acc(torch.sigmoid(pred), label.int().unsqueeze(1)))

  
  def validation_epoch_end(self, outs):
    self.log("Val ACC", self.val_acc.compute())

  def configure_optimizers(self):
      return [self.optimizer]

#Model Training

In [12]:
def evaluation(model, dataloader):
  predictions, labels = [], []
  with torch.no_grad():
    for batch in dataloader:
      batch = [term.cuda() for term in batch]
      pred = model(batch[0]).squeeze(1)
      pred = torch.sigmoid(pred)
      pred = [1 if p >= 0.5 else 0 for p in pred]
      predictions += pred
      labels += batch[1]
  
  result = 0
  for p, l in zip(predictions, labels):

    if p == l:
      result += 1
    else:
      continue
  accuracy = result / len(labels)
  return accuracy

In [ ]:
import tqdm
best_accuracy = float('-inf')
# state_dict = torch.load('file_path')
# model.load_state_dict(state_dict)
step = 0
patinet = 0
stop_train = False
model = CheastCancer_ConvNet()
model = model.cuda()
for epoch in range(50):
  bar = tqdm.trange(len(train_loader))
  total_loss = 0
  for epoch_step, batch in zip(bar, train_loader):
    model.optimizer.zero_grad()
    model.train()
    batch = [term.cuda() for term in batch]

    pred = model(batch[0])
    loss = model.criterion(pred.logits,batch[1].float().unsqueeze(1))
    # loss = model.criterion(pred.squeeze(1), batch[1].float())

    loss.backward()
    total_loss += loss.item()
    model.optimizer.step()
    bar.set_postfix(avg_loss='{}'.format(total_loss / (epoch_step+1)))

    if step % 100 == 0 and step != 0:
      model.eval()
      val_accu = evaluation(model, val_loader)
      print('Val Accuracy: {}'.format(val_accu))

      if val_accu >= best_accuracy:
        torch.save(model.state_dict(), '/content/drive/MyDrive/Project/checkpoint/Inception/checkpoint.ckpt')
        patient = 0
      else:
        patient += 1
      
      if patient == 5:
        stop_train = True
        break
    step += 1
  if stop_train:
    print('Stop Training !')
    break

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:48: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,
  4%|▍         | 100/2388 [04:43<1:52:50,  2.96s/it, avg_loss=0.7439614298320053]

Val Accuracy: 0.5521637971149371


  8%|▊         | 200/2388 [10:49<1:46:16,  2.91s/it, avg_loss=0.7248245711943403]

Val Accuracy: 0.5174499767333643


 13%|█▎        | 300/2388 [16:55<1:44:19,  3.00s/it, avg_loss=0.72036320485942]  

...

  2%|▏         | 44/2388 [00:09<08:12,  4.76it/s, avg_loss=0.6902889794773526]

Val Accuracy: 0.5663099115867846


  6%|▌         | 144/2388 [01:45<07:43,  4.84it/s, avg_loss=0.6904308865810262]

Val Accuracy: 0.5781293624941833


  7%|▋         | 173/2388 [03:06<07:37,  4.84it/s, avg_loss=0.6952168710659005]